# This is notebook refers a education project to evatuate a credit for a fictional company and this is a part of Udemy's course "Formação Cientista de Dados com Python e R" by Fernando Amaral.
## Author Wagner Brito.

## Part 1 - Getting data

In [3]:
import psycopg2
import pandas as pd

In [2]:
# open a database conection
conexao = psycopg2.connect(host="localhost",database="wbk_db", user="postgres", password="postgres", port=5432)

In [3]:
'''
Some "constanst" to help querys
'''
sql_tables = "SELECT table_name FROM information_schema.tables WHERE table_schema = %s"
str_schema = 'public'
columns = 'COLUMNS'
data = 'DATA'

In [4]:
'''
Method to recovery all data from specific query
'''
def return_data(conn,sql,params):
    cursor = conn.cursor()
    cursor.execute(sql,params)
    retorno = cursor.fetchall()
    cursor.close()
    return retorno

In [5]:
'''
Change tuple in list 
'''
def return_list_from_record(data):
    retorno = []
    for d in data:
        if len(d) == 1:
            retorno.append(d[0])
        else:
            inner_list = []
            for d1 in d:
                inner_list.append(d1)
            retorno.append(inner_list)
    return retorno

In [6]:
'''
Auxiliar method to recovery columns from specific table
'''
def return_table_columns(conn,table_name):
    sql_columns = "SELECT column_name FROM information_schema.columns WHERE table_schema=%s AND table_name = %s"
    return return_data(conn,sql_columns,[str_schema,table_name])

In [7]:
'''
Make a SQL based in columns list and a table name
'''
def make_query(colunms,table):
    return 'SELECT '+', '.join(colunms)+" FROM "+table

In [8]:
'''
Make a dictionary with key call 'DATA' and data is a Pandas DataFrame with all records from a table name
'''
def get_dataframe_from_table(conexao,table_name):
    columns_list = return_list_from_record(return_table_columns(conexao,table_name))
    str_sql = make_query(columns_list,table_name)
    table_data = return_data(conexao,str_sql,[])
    return pd.DataFrame(table_data, columns = columns_list)

In [9]:
'''
Make a dictionary with this structure:
    'TABLE_NAME':{
        'COLUMNS':[List of columns names],
        'DATA':Pandas datafreme
    }
'''
def create_data_dict():
    dict_out = {}
    table_list = return_list_from_record(return_data(conexao,sql_tables,[str_schema]))
    for t in table_list:
        dict_out[t] = {
            columns:return_list_from_record(return_table_columns(conexao,t)),
            data:get_dataframe_from_table(conexao,t)
        } 
    return dict_out

In [10]:
data_dict = create_data_dict()

In [11]:
data_dict

{'clientes': {'COLUMNS': ['idcliente', 'cliente', 'estado', 'sexo'],
  'DATA':      idcliente               cliente estado sexo
  0            1  Adelina Buenaventura     RJ    M
  1            2          Adelino Gago     RJ    M
  2            3         Adélio Lisboa     SE    M
  3            4         Adérito Bahía     MA    M
  4            5       Adolfo Patrício     PE    M
  ..         ...                   ...    ...  ...
  245        246  Jandaíra Albuquerque     SP    F
  246        247            Joana Ataí     GO    F
  247        248       Joaquim Hurtado     AP    M
  248        249        Joaquim Mieiro     TO    M
  249        250  Joaquina Vasconcelos     SC    F
  
  [250 rows x 4 columns]},
 'credito': {'COLUMNS': ['idcredito',
   'duracao',
   'historicocredito',
   'proposito',
   'valor',
   'investimentos',
   'emprego',
   'tempoparcelamento',
   'estadocivil',
   'fiadorterceiros',
   'residenciadesde',
   'idade',
   'outrosfinanciamentos',
   'habitacao',
   

In [11]:
conexao.close()

In [12]:
'''
Looking for a similar columns in all tables
'''
for key in data_dict:
    #print(key)
    print(data_dict[key][columns])

['idcliente', 'cliente', 'estado', 'sexo']
['idcredito', 'duracao', 'historicocredito', 'proposito', 'valor', 'investimentos', 'emprego', 'tempoparcelamento', 'estadocivil', 'fiadorterceiros', 'residenciadesde', 'idade', 'outrosfinanciamentos', 'habitacao', 'emprestimoexistente', 'profissao', 'dependentes', 'socioempresa', 'estrangeiro', 'status']
['idemprego', 'emprego']
['idestadocivil', 'estadocivil']
['idfiador', 'fiador']
['idhabitacao', 'habitacao']
['idhistcred', 'historico']
['idinvestimentos', 'investimentos']
['idoutrosfinanc', 'outrosfinanciamentos']
['idprofissao', 'profissao']
['idproposito', 'proposito']


In [14]:
for key in data_dict:
    print('-------------------------------------------------------------')
    print(key)
    print(data_dict[key][data])
    print('-------------------------------------------------------------')  

-------------------------------------------------------------
clientes
                        cliente estado sexo
idcliente                                  
1          Adelina Buenaventura     RJ    M
2                  Adelino Gago     RJ    M
3                 Adélio Lisboa     SE    M
4                 Adérito Bahía     MA    M
5               Adolfo Patrício     PE    M
...                         ...    ...  ...
246        Jandaíra Albuquerque     SP    F
247                  Joana Ataí     GO    F
248             Joaquim Hurtado     AP    M
249              Joaquim Mieiro     TO    M
250        Joaquina Vasconcelos     SC    F

[250 rows x 3 columns]
-------------------------------------------------------------
-------------------------------------------------------------
credito
           duracao  historicocredito  proposito  valor  investimentos  \
idcredito                                                               
2               48                 2          1   5951 

All DataFrame that I created have a 'extra' index because my routines put all 'id' columns in a 
DataFrame column. Now I'll try to reindex this.

In [12]:
# Reindex all data frame 
for key in data_dict:
    data_dict[key][data].set_index(data_dict[key][data].columns[0],inplace=True) 

## Part 2 - Exploring data

In [13]:
data_dict.keys()

dict_keys(['clientes', 'credito', 'emprego', 'estadocivil', 'fiador', 'habitacao', 'historico_credito', 'investimentos', 'outrosfinanc', 'profissao', 'proposito'])

In [14]:
credito = data_dict['credito'][data]

In [17]:
credito.head()

,duracao,historicocredito,proposito,valor,investimentos,emprego,tempoparcelamento,estadocivil,fiadorterceiros,residenciadesde,idade,outrosfinanciamentos,habitacao,emprestimoexistente,profissao,dependentes,socioempresa,estrangeiro,status
idcredito,,,,,,,,,,,,,,,,,,,
2,48,2,1,5951,2,3.0,2,1,3,2.0,22,3,1.0,1,4,1,0,0,0
3,12,1,2,2096,2,4.0,2,4,3,3.0,49,3,1.0,1,3,2,0,0,1
4,42,2,3,7882,2,4.0,2,4,4,4.0,45,3,2.0,1,4,2,0,0,1
5,24,3,4,4870,2,3.0,3,4,3,4.0,53,3,2.0,2,4,2,0,0,0
6,36,2,2,9055,1,3.0,2,4,3,4.0,35,3,2.0,1,3,2,1,0,1


In [15]:
credito.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 2 to 565
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   duracao               1000 non-null   int64  
 1   historicocredito      1000 non-null   int64  
 2   proposito             1000 non-null   int64  
 3   valor                 1000 non-null   int64  
 4   investimentos         1000 non-null   int64  
 5   emprego               990 non-null    float64
 6   tempoparcelamento     1000 non-null   int64  
 7   estadocivil           1000 non-null   int64  
 8   fiadorterceiros       1000 non-null   int64  
 9   residenciadesde       993 non-null    float64
 10  idade                 1000 non-null   int64  
 11  outrosfinanciamentos  1000 non-null   int64  
 12  habitacao             991 non-null    float64
 13  emprestimoexistente   1000 non-null   int64  
 14  profissao             1000 non-null   int64  
 15  dependentes           

In first look I have: <br>
10 records without emprego; <br>
7 records without residenciadesde; <br>
9 records without habitacao.

In [16]:
# About emprego
emprego = data_dict['emprego'][data]

In [17]:
emprego

,emprego
idemprego,
1,<1
2,>=7
3,1<=X<4
4,4<=X<7
5,Desempregado


In [18]:
credito['emprego'].unique()

array([ 3.,  4.,  2.,  1.,  5., nan])

In [19]:
# verify how emprego are distributed
job_group = credito.groupby(['emprego']).size()
job_group

emprego
1.0    171
2.0    247
3.0    338
4.0    172
5.0     62
dtype: int64

In [20]:
credito[credito['emprego'].isnull()]

,duracao,historicocredito,proposito,valor,investimentos,emprego,tempoparcelamento,estadocivil,fiadorterceiros,residenciadesde,idade,outrosfinanciamentos,habitacao,emprestimoexistente,profissao,dependentes,socioempresa,estrangeiro,status
idcredito,,,,,,,,,,,,,,,,,,,
1,6,0,1,1169,1,NaN,4,4,3,4.0,67,3,1.0,2,4,1,1,0,1
78,11,3,1,4771,2,NaN,2,4,3,4.0,51,3,1.0,1,4,1,0,0,1
145,21,1,3,2288,2,NaN,4,1,3,4.0,23,3,NaN,1,4,1,1,0,1
960,24,2,3,3069,5,NaN,4,4,3,NaN,30,3,2.0,1,4,1,0,0,1
56,6,5,4,783,1,NaN,1,4,4,2.0,26,2,1.0,1,3,2,0,0,1
270,24,2,1,999,1,NaN,4,4,3,2.0,25,3,1.0,2,4,1,0,0,1
896,36,3,5,8947,1,NaN,3,4,3,2.0,31,2,1.0,1,1,2,1,0,1
199,24,2,5,2760,1,NaN,4,4,3,4.0,36,1,NaN,1,4,1,1,0,1
45,48,1,5,6143,2,NaN,4,1,3,4.0,58,2,2.0,2,999,1,0,0,0


Besides the emprego is null the profissao was filled so I decide to grap the average of each profissao to fill emprego column.

In [21]:
credito[credito['emprego'].isnull()]['profissao']

idcredito
1        4
78       4
145      4
960      4
56       3
270      4
896      1
199      4
45     999
95     999
Name: profissao, dtype: int64

In [22]:
avg_emprego_4 = credito[credito['profissao'] == 4].groupby(['emprego']).size()

In [23]:
avg_emprego_4

emprego
1.0    105
2.0    156
3.0    228
4.0    118
5.0     12
dtype: int64

In [24]:
avg_emprego_3 = credito[credito['profissao'] == 3].groupby(['emprego']).size()

In [25]:
avg_emprego_3

emprego
1.0    44
2.0    39
3.0    78
4.0    34
5.0     1
dtype: int64

In [26]:
avg_emprego_1 = credito[credito['profissao'] == 1].groupby(['emprego']).size()

In [27]:
avg_emprego_1

emprego
1.0    16
2.0    50
3.0    28
4.0    19
5.0    33
dtype: int64

In [28]:
avg_emprego_999 = credito[credito['profissao'] == 999].groupby(['emprego']).size()

In [29]:
avg_emprego_999

emprego
1.0    1
2.0    2
3.0    3
4.0    1
dtype: int64

After some obsrevations we have:<br>
- profissao 999, 3 and 4 => max(emprego) == 3<br>
- profissao 1 => max(emprego) == 2

In [30]:
credito['emprego'].isnull().values.any()

True

#### This is the right way to correcting 'emprego' missing data

In [31]:
credito.loc[(credito['profissao'] == 3) & (credito['emprego'].isnull()),'emprego'] = 3.0

In [32]:
credito.loc[(credito['profissao'] == 4) & (credito['emprego'].isnull()),'emprego'] = 3.0

In [33]:
credito.loc[(credito['profissao'] == 999) & (credito['emprego'].isnull()),'emprego'] = 3.0

In [34]:
credito.loc[(credito['profissao'] == 1) & (credito['emprego'].isnull()),'emprego'] = 2.0

In [35]:
credito.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 2 to 565
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   duracao               1000 non-null   int64  
 1   historicocredito      1000 non-null   int64  
 2   proposito             1000 non-null   int64  
 3   valor                 1000 non-null   int64  
 4   investimentos         1000 non-null   int64  
 5   emprego               1000 non-null   float64
 6   tempoparcelamento     1000 non-null   int64  
 7   estadocivil           1000 non-null   int64  
 8   fiadorterceiros       1000 non-null   int64  
 9   residenciadesde       993 non-null    float64
 10  idade                 1000 non-null   int64  
 11  outrosfinanciamentos  1000 non-null   int64  
 12  habitacao             991 non-null    float64
 13  emprestimoexistente   1000 non-null   int64  
 14  profissao             1000 non-null   int64  
 15  dependentes           

In [57]:
credito[credito['residenciadesde'].isnull()]['idade']

idcredito
35     33
15     28
200    40
75     40
350    32
650    40
960    30
Name: idade, dtype: int64

In [56]:
residenciadesde_group = credito.groupby(['residenciadesde']).size()
residenciadesde_group

residenciadesde
1.0    129
2.0    308
3.0    147
4.0    409
dtype: int64

In [36]:
credito['residenciadesde'].fillna(4.0, inplace=True)

In [50]:
credito.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 2 to 565
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   duracao               1000 non-null   int64  
 1   historicocredito      1000 non-null   int64  
 2   proposito             1000 non-null   int64  
 3   valor                 1000 non-null   int64  
 4   investimentos         1000 non-null   int64  
 5   emprego               1000 non-null   float64
 6   tempoparcelamento     1000 non-null   int64  
 7   estadocivil           1000 non-null   int64  
 8   fiadorterceiros       1000 non-null   int64  
 9   residenciadesde       1000 non-null   float64
 10  idade                 1000 non-null   int64  
 11  outrosfinanciamentos  1000 non-null   int64  
 12  habitacao             991 non-null    float64
 13  emprestimoexistente   1000 non-null   int64  
 14  profissao             1000 non-null   int64  
 15  dependentes           

In [60]:
credito[credito['habitacao'].isnull()]

,duracao,historicocredito,proposito,valor,investimentos,emprego,tempoparcelamento,estadocivil,fiadorterceiros,residenciadesde,idade,outrosfinanciamentos,habitacao,emprestimoexistente,profissao,dependentes,socioempresa,estrangeiro,status
idcredito,,,,,,,,,,,,,,,,,,,
8,36,2,5,6948,2,3.0,2,4,3,2.0,35,3,NaN,1,1,1,1,0,1
12,48,2,6,4308,2,1.0,3,1,3,4.0,24,3,NaN,1,4,1,0,0,0
145,21,1,3,2288,2,3.0,4,1,3,4.0,23,3,NaN,1,4,1,1,0,1
945,15,2,3,1845,2,1.0,4,1,4,1.0,46,3,NaN,1,4,1,0,0,1
199,24,2,5,2760,1,3.0,4,4,3,4.0,36,1,NaN,1,4,1,1,0,1
356,24,2,4,1246,2,1.0,4,4,3,2.0,23,2,NaN,1,3,1,0,0,0
396,39,3,2,11760,5,4.0,2,4,3,3.0,32,3,NaN,1,4,1,1,0,1
745,39,1,3,14179,1,4.0,4,4,3,4.0,30,3,NaN,2,1,1,1,0,1
750,15,2,5,3029,2,4.0,2,4,3,2.0,33,3,NaN,1,4,1,0,0,1


In [61]:
credito['habitacao'].unique()

array([ 1.,  2.,  3., nan])

In [62]:
habitacao_group = credito.groupby(['habitacao']).size()
habitacao_group

habitacao
1.0    709
2.0    107
3.0    175
dtype: int64

In [37]:
credito['habitacao'].fillna(1.0, inplace=True)

In [52]:
credito.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 2 to 565
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   duracao               1000 non-null   int64  
 1   historicocredito      1000 non-null   int64  
 2   proposito             1000 non-null   int64  
 3   valor                 1000 non-null   int64  
 4   investimentos         1000 non-null   int64  
 5   emprego               1000 non-null   float64
 6   tempoparcelamento     1000 non-null   int64  
 7   estadocivil           1000 non-null   int64  
 8   fiadorterceiros       1000 non-null   int64  
 9   residenciadesde       1000 non-null   float64
 10  idade                 1000 non-null   int64  
 11  outrosfinanciamentos  1000 non-null   int64  
 12  habitacao             1000 non-null   float64
 13  emprestimoexistente   1000 non-null   int64  
 14  profissao             1000 non-null   int64  
 15  dependentes           

In [53]:
'''
See the values of status our class to infer
'''
credito['status'].unique()

array([0, 1])

#### Now I see for relations between credito dataset and othes data tables if that something filled wrong 

In [54]:
data_dict.keys()

dict_keys(['clientes', 'credito', 'emprego', 'estadocivil', 'fiador', 'habitacao', 'historico_credito', 'investimentos', 'outrosfinanc', 'profissao', 'proposito'])

In [57]:
credito.columns

Index(['duracao', 'historicocredito', 'proposito', 'valor', 'investimentos',
       'emprego', 'tempoparcelamento', 'estadocivil', 'fiadorterceiros',
       'residenciadesde', 'idade', 'outrosfinanciamentos', 'habitacao',
       'emprestimoexistente', 'profissao', 'dependentes', 'socioempresa',
       'estrangeiro', 'status'],
      dtype='object')

In [38]:
credito['historicocredito'].unique()

array([2, 1, 3, 4, 5, 0])

In [60]:
data_dict['historico_credito']

{'COLUMNS': ['idhistcred', 'historico'],
 'DATA':                            historico
 idhistcred                          
 1            Critico-outros creditos
 2                   existentes/pagos
 3                 Atrasos anteriores
 4           sem creditos/todos pagos
 5                        todos pagos}

#### 0 is not a valid value for 'historico_credito' 

In [62]:
credito[credito['historicocredito'] == 0]

,duracao,historicocredito,proposito,valor,investimentos,emprego,tempoparcelamento,estadocivil,fiadorterceiros,residenciadesde,idade,outrosfinanciamentos,habitacao,emprestimoexistente,profissao,dependentes,socioempresa,estrangeiro,status
idcredito,,,,,,,,,,,,,,,,,,,
1,6,0,1,1169,1,3.0,4,4,3,4.0,67,3,1.0,2,4,1,1,0,1
10,30,0,4,5234,2,5.0,4,3,3,2.0,28,3,1.0,2,1,1,0,0,0
169,24,0,3,3972,2,4.0,2,1,3,4.0,25,3,3.0,1,4,1,1,0,1
233,12,0,1,2141,5,4.0,3,4,3,1.0,35,3,1.0,1,4,1,0,0,1
395,9,0,3,2406,2,5.0,2,4,3,3.0,31,3,1.0,1,1,1,0,0,1
676,30,0,1,4530,2,4.0,4,1,3,4.0,26,3,3.0,1,1,1,1,0,1
799,24,0,4,717,1,2.0,4,3,3,4.0,54,3,1.0,2,4,1,1,0,1
844,24,0,6,1559,2,4.0,4,4,3,4.0,50,1,1.0,1,4,1,1,0,1
234,18,0,1,866,2,3.0,4,3,4,2.0,25,3,1.0,1,3,1,0,0,1


In [63]:
historicocredito_group = credito.groupby(['historicocredito']).size()
historicocredito_group

historicocredito
0     11
1    289
2    526
3     86
4     40
5     48
dtype: int64

The most part of data consists in type 2 existentes/pagos in 0 (zeros) cases only one (id = 10) is considered bad credit so I decide to replace most part of 0 to 2. The id = 10 I'll replace by type 1 Critico-outros creditos

In [39]:
credito.loc[(credito['historicocredito'] == 0) & (credito['status'] == 1),'historicocredito'] = 2

In [40]:
credito.loc[(credito['historicocredito'] == 0) & (credito['status'] == 0),'historicocredito'] = 1

In [41]:
credito[credito['historicocredito'] == 0]

,duracao,historicocredito,proposito,valor,investimentos,emprego,tempoparcelamento,estadocivil,fiadorterceiros,residenciadesde,idade,outrosfinanciamentos,habitacao,emprestimoexistente,profissao,dependentes,socioempresa,estrangeiro,status
idcredito,,,,,,,,,,,,,,,,,,,


In [69]:
credito['proposito'].unique()

array([ 1,  2,  3,  4,  6,  5,  7,  8,  9, 10])

In [70]:
data_dict['proposito']

{'COLUMNS': ['idproposito', 'proposito'],
 'DATA':                  proposito
 idproposito               
 1                  reforma
 2                 educacao
 3                   moveis
 4             veiculo novo
 5            veiculo usado
 6            abrir negocio
 7            quitar divida
 8             reforma casa
 9                   outros
 10            qualificacao}

'proposito' is Ok

In [71]:
credito['investimentos'].unique()

array([2, 1, 3, 4, 5])

In [72]:
data_dict['investimentos']

{'COLUMNS': ['idinvestimentos', 'investimentos'],
 'DATA':                   investimentos
 idinvestimentos                
 1                Desconhecido\n
 2                        <100\n
 3                   500<=X<1000
 4                        >=1000
 5                    100<=X<500}

'investimentos' is Ok

In [73]:
credito['emprego'].unique()

array([3., 4., 2., 1., 5.])

In [74]:
data_dict['emprego']

{'COLUMNS': ['idemprego', 'emprego'],
 'DATA':                 emprego
 idemprego              
 1                    <1
 2                   >=7
 3                1<=X<4
 4                4<=X<7
 5          Desempregado}

'emprego' is Ok

In [76]:
data_dict['estadocivil']

{'COLUMNS': ['idestadocivil', 'estadocivil'],
 'DATA':                         estadocivil
 idestadocivil                      
 1              feminino div/dep/cas
 2                  maculino div/dep
 3                 masculino cas/viu
 4                masculino solteiro}

In [75]:
credito['estadocivil'].unique()

array([1, 4, 2, 3])

'estadocivil' is Ok

In [79]:
credito['fiadorterceiros'].unique()

array([3, 4, 2, 1])

In [77]:
data_dict['fiador']

{'COLUMNS': ['idfiador', 'fiador'],
 'DATA':                  fiador
 idfiador               
 1          co aplicante
 2         co requerente
 3                   nao
 4                   sim}

#### If fiador is the same that fiadorterceiros is Ok

In [80]:
credito['outrosfinanciamentos'].unique()

array([3, 1, 2])

In [81]:
data_dict['outrosfinanc']

{'COLUMNS': ['idoutrosfinanc', 'outrosfinanciamentos'],
 'DATA':                outrosfinanciamentos
 idoutrosfinanc                     
 1                             banco
 2                              loja
 3                            nenhum}

#### If outrosfinanciamentos and outrosfinanc is the same, Ok.

In [57]:
credito.columns

Index(['duracao', 'historicocredito', 'proposito', 'valor', 'investimentos',
       'emprego', 'tempoparcelamento', 'estadocivil', 'fiadorterceiros',
       'residenciadesde', 'idade', 'outrosfinanciamentos', 'habitacao',
       'emprestimoexistente', 'profissao', 'dependentes', 'socioempresa',
       'estrangeiro', 'status'],
      dtype='object')

In [54]:
data_dict.keys()

dict_keys(['clientes', 'credito', 'emprego', 'estadocivil', 'fiador', 'habitacao', 'historico_credito', 'investimentos', 'outrosfinanc', 'profissao', 'proposito'])

In [82]:
credito['profissao'].unique()

array([  4,   3,   1,   2, 999])

In [83]:
data_dict['profissao']

{'COLUMNS': ['idprofissao', 'profissao'],
 'DATA':                                     profissao
 idprofissao                                  
 1            altamente qualificado/empresario
 2                                desempregado
 3                             nao qualificado
 4                                 qualificado}

### Remain some credito data with wrong profession

In [42]:
credito[credito['profissao'] == 999]

,duracao,historicocredito,proposito,valor,investimentos,emprego,tempoparcelamento,estadocivil,fiadorterceiros,residenciadesde,idade,outrosfinanciamentos,habitacao,emprestimoexistente,profissao,dependentes,socioempresa,estrangeiro,status
idcredito,,,,,,,,,,,,,,,,,,,
69,36,2,2,1819,2,3.0,4,4,3,4.0,37,2,2.0,1,999,1,1,0,0
174,8,2,1,1414,2,3.0,4,4,4,2.0,33,3,1.0,1,999,1,0,1,1
412,33,1,5,7253,2,4.0,3,4,3,2.0,35,3,1.0,2,999,1,1,0,1
976,24,2,1,1258,3,3.0,3,1,3,3.0,57,3,1.0,1,999,1,0,0,1
45,48,1,5,6143,2,3.0,4,1,3,4.0,58,2,2.0,2,999,1,0,0,0
95,12,2,4,1318,4,3.0,4,4,3,4.0,54,3,1.0,1,999,1,1,0,1
499,18,2,1,1301,2,2.0,4,3,4,2.0,32,3,1.0,1,999,1,0,0,1
505,24,2,4,1207,2,1.0,4,1,3,4.0,24,3,3.0,1,999,1,0,0,0
834,24,2,1,5084,1,2.0,2,1,3,4.0,42,3,1.0,1,999,1,1,0,1


In [85]:
profissao_group = credito.groupby(['profissao']).size()
profissao_group

profissao
1      147
2       22
3      197
4      625
999      9
dtype: int64

The most part of credito data have profissao = 4 qualificado but I'll look more close how many this records received good credit

In [87]:
profissao_group_2 = credito[credito['profissao'] == 4].groupby(['status']).count()
profissao_group_2

,duracao,historicocredito,proposito,valor,investimentos,emprego,tempoparcelamento,estadocivil,fiadorterceiros,residenciadesde,idade,outrosfinanciamentos,habitacao,emprestimoexistente,profissao,dependentes,socioempresa,estrangeiro
status,,,,,,,,,,,,,,,,,,
0,184,184,184,184,184,184,184,184,184,184,184,184,184,184,184,184,184,184
1,441,441,441,441,441,441,441,441,441,441,441,441,441,441,441,441,441,441


The distribuiton look's like Ok. I decide change all profissao = 999 to 4

In [43]:
credito.loc[(credito['profissao'] == 999),'profissao'] = 4

In [44]:
credito[credito['profissao'] == 999]

,duracao,historicocredito,proposito,valor,investimentos,emprego,tempoparcelamento,estadocivil,fiadorterceiros,residenciadesde,idade,outrosfinanciamentos,habitacao,emprestimoexistente,profissao,dependentes,socioempresa,estrangeiro,status
idcredito,,,,,,,,,,,,,,,,,,,


In [45]:
'''
Percist a cleanup version of dataset
'''
credito.to_csv('credito_01.csv')

### Now I decide transform some continuos data in categorical like 'duracao', 'valor', 'idade' and others if necessary

In [6]:
credito = pd.read_csv('credito_01.csv')

In [14]:
'''
Looking 'duracao' values
'''
credito['duracao'].unique()

array([3, 1, 2, 4])

In [10]:
credito['duracao'].min()

4

In [12]:
credito['duracao'].quantile(0.25)

12.0

In [14]:
credito['duracao'].median()

18.0

In [13]:
credito['duracao'].quantile(0.75)

24.0

In [11]:
credito['duracao'].max()

72

In [8]:
'''
Based in amplitude and range of values I'll make 'duracao' in these classes
1 :  <= 21
2 :  > 21 and <= 38
3 :  > 38 and <= 55
4 :  > 55
'''
def duration_class(duration):
    if duration <= 21:
        return 1
    elif duration > 21 and duration <= 38:
        return 2
    elif duration > 38 and duration <= 55:
        return 3
    else:
        return 4

In [9]:
credito['duracao'] = credito['duracao'].apply(duration_class)

In [49]:
'''
Looking 'idade' values
'''
credito['idade'].min()

19

In [50]:
credito['idade'].quantile(0.25)

27.0

In [53]:
credito['idade'].median()

33.0

In [51]:
credito['idade'].quantile(0.75)

42.0

In [52]:
credito['idade'].max()

75

In [54]:
credito['idade'].max() - credito['idade'].min()

56

In [55]:
(credito['idade'].max() - credito['idade'].min())/4

14.0

In [11]:
'''
Based in amplitude and range of values I'll make 'idade' in these classes
1 :  <= 33
2 :   > 33 and <= 47
3 :   > 47 and <= 61
4 :   > 61
'''
def idade_class(age):
    if age <= 33:
        return 1
    elif age > 33 and age <= 47:
        return 2
    elif age > 47 and age <= 61:
        return 3
    else:
        return 4

In [12]:
credito['idade'] = credito['idade'].apply(idade_class)

In [13]:
credito['idade'].unique()

array([1, 3, 2, 4])

In [15]:
'''
Percist a categorigal version of dataset
'''
credito.to_csv('credito_02.csv')

## Part 3 - Model creation

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

In [5]:
'''
Loading all versions of credito to evaluate all posible results 
'''
credito_01 = pd.read_csv('credito_01.csv',index_col='idcredito')
credito_02 = pd.read_csv('credito_02.csv',index_col='idcredito')
credito_03 = pd.read_csv('credito_03.csv')

In [6]:
credito_01.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 2 to 565
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   duracao               1000 non-null   int64  
 1   historicocredito      1000 non-null   int64  
 2   proposito             1000 non-null   int64  
 3   valor                 1000 non-null   int64  
 4   investimentos         1000 non-null   int64  
 5   emprego               1000 non-null   float64
 6   tempoparcelamento     1000 non-null   int64  
 7   estadocivil           1000 non-null   int64  
 8   fiadorterceiros       1000 non-null   int64  
 9   residenciadesde       1000 non-null   float64
 10  idade                 1000 non-null   int64  
 11  outrosfinanciamentos  1000 non-null   int64  
 12  habitacao             1000 non-null   float64
 13  emprestimoexistente   1000 non-null   int64  
 14  profissao             1000 non-null   int64  
 15  dependentes           

In [7]:
credito_02.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 2 to 565
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            1000 non-null   int64  
 1   duracao               1000 non-null   int64  
 2   historicocredito      1000 non-null   int64  
 3   proposito             1000 non-null   int64  
 4   valor                 1000 non-null   int64  
 5   investimentos         1000 non-null   int64  
 6   emprego               1000 non-null   float64
 7   tempoparcelamento     1000 non-null   int64  
 8   estadocivil           1000 non-null   int64  
 9   fiadorterceiros       1000 non-null   int64  
 10  residenciadesde       1000 non-null   float64
 11  idade                 1000 non-null   int64  
 12  outrosfinanciamentos  1000 non-null   int64  
 13  habitacao             1000 non-null   float64
 14  emprestimoexistente   1000 non-null   int64  
 15  profissao             

In [10]:
credito_02.columns[0]

'Unnamed: 0'

In [11]:
'''
Removing Unnamed because is not necessary for modeling purposes
'''
credito_02.drop(axis=1,columns=credito_02.columns[0],inplace=True)

In [12]:
credito_02.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 2 to 565
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   duracao               1000 non-null   int64  
 1   historicocredito      1000 non-null   int64  
 2   proposito             1000 non-null   int64  
 3   valor                 1000 non-null   int64  
 4   investimentos         1000 non-null   int64  
 5   emprego               1000 non-null   float64
 6   tempoparcelamento     1000 non-null   int64  
 7   estadocivil           1000 non-null   int64  
 8   fiadorterceiros       1000 non-null   int64  
 9   residenciadesde       1000 non-null   float64
 10  idade                 1000 non-null   int64  
 11  outrosfinanciamentos  1000 non-null   int64  
 12  habitacao             1000 non-null   float64
 13  emprestimoexistente   1000 non-null   int64  
 14  profissao             1000 non-null   int64  
 15  dependentes           

In [13]:
credito_03.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            1000 non-null   int64  
 1   duracao               1000 non-null   int64  
 2   historicocredito      1000 non-null   int64  
 3   proposito             1000 non-null   int64  
 4   valor                 1000 non-null   float64
 5   investimentos         1000 non-null   int64  
 6   emprego               1000 non-null   float64
 7   tempoparcelamento     1000 non-null   int64  
 8   estadocivil           1000 non-null   int64  
 9   fiadorterceiros       1000 non-null   int64  
 10  residenciadesde       1000 non-null   float64
 11  idade                 1000 non-null   int64  
 12  outrosfinanciamentos  1000 non-null   int64  
 13  habitacao             1000 non-null   float64
 14  emprestimoexistente   1000 non-null   int64  
 15  profissao             

In [14]:
'''
Removing Unnamed because is not necessary for modeling purposes
'''
credito_03.drop(axis=1,columns=credito_03.columns[0],inplace=True)

In [15]:
credito_03.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   duracao               1000 non-null   int64  
 1   historicocredito      1000 non-null   int64  
 2   proposito             1000 non-null   int64  
 3   valor                 1000 non-null   float64
 4   investimentos         1000 non-null   int64  
 5   emprego               1000 non-null   float64
 6   tempoparcelamento     1000 non-null   int64  
 7   estadocivil           1000 non-null   int64  
 8   fiadorterceiros       1000 non-null   int64  
 9   residenciadesde       1000 non-null   float64
 10  idade                 1000 non-null   int64  
 11  outrosfinanciamentos  1000 non-null   int64  
 12  habitacao             1000 non-null   float64
 13  emprestimoexistente   1000 non-null   int64  
 14  profissao             1000 non-null   int64  
 15  dependentes           

I'll run the attribute selector ExtraTreesClassifier and compare models generate using all dataset attibutes with models attibutes choice using ExtraTreesClassifier

In [16]:
'''
Separating predictors and the target value for classification
'''
previsores_01 = credito_01.iloc[:,0:18].values
classe_01 = credito_01.iloc[:,18].values
previsores_02 = credito_02.iloc[:,0:18].values
classe_02 = credito_02.iloc[:,18].values
previsores_03 = credito_03.iloc[:,0:18].values
classe_03 = credito_03.iloc[:,18].values

In [17]:
# Spliting data
X_treinamento_01, X_teste_01, y_treinamento_01, y_teste_01 = train_test_split(previsores_01,
                                                                  classe_01,
                                                                  test_size = 0.3,
                                                                  random_state = 1)
X_treinamento_02, X_teste_02, y_treinamento_02, y_teste_02 = train_test_split(previsores_02,
                                                                  classe_02,
                                                                  test_size = 0.3,
                                                                  random_state = 1)
X_treinamento_03, X_teste_03, y_treinamento_03, y_teste_03 = train_test_split(previsores_03,
                                                                  classe_03,
                                                                  test_size = 0.3,
                                                                  random_state = 1)

In [18]:
'''
Using a ExtraTreesClassifier to help evaluating 'credito' attributes.
'''
forest = ExtraTreesClassifier()
forest.fit(X_treinamento_01, y_treinamento_01)
importancias_01 = forest.feature_importances_
indices_01 = np.where(importancias_01 > 0.05)
forest = ExtraTreesClassifier()
forest.fit(X_treinamento_02, y_treinamento_02)
importancias_02 = forest.feature_importances_
indices_02 = np.where(importancias_02 > 0.05)
forest = ExtraTreesClassifier()
forest.fit(X_treinamento_03, y_treinamento_03)
importancias_03 = forest.feature_importances_
indices_03 = np.where(importancias_03 > 0.05)

In [19]:
indices_01

(array([ 0,  1,  2,  3,  4,  5,  6,  9, 10]),)

In [20]:
indices_02

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  9, 10, 14]),)

In [21]:
indices_03

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  9, 10, 14]),)

### First algorithm Support Vector Machine

In [23]:
svm_selected_01 = SVC()
svm_selected_01.fit(X_treinamento_01[:,indices_01[0]], y_treinamento_01)
previsoes_01 = svm_selected_01.predict(X_teste_01[:,indices_01[0]])
taxa_acerto_01 = accuracy_score(y_teste_01, previsoes_01)
taxa_acerto_01

0.7066666666666667

In [24]:
svm_selected_02 = SVC()
svm_selected_02.fit(X_treinamento_02[:,indices_02[0]], y_treinamento_02)
previsoes_02 = svm_selected_02.predict(X_teste_02[:,indices_02[0]])
taxa_acerto_02 = accuracy_score(y_teste_02, previsoes_02)
taxa_acerto_02

0.7066666666666667

In [25]:
svm_selected_03 = SVC()
svm_selected_03.fit(X_treinamento_03[:,indices_03[0]], y_treinamento_03)
previsoes_03 = svm_selected_03.predict(X_teste_03[:,indices_03[0]])
taxa_acerto_03 = accuracy_score(y_teste_03, previsoes_03)
taxa_acerto_03

0.7133333333333334

In [27]:
svm_complete_01 = SVC()
svm_complete_01.fit(X_treinamento_01, y_treinamento_01)
previsoes_01 = svm_complete_01.predict(X_teste_01)
taxa_acerto = accuracy_score(y_teste_01, previsoes_01)
taxa_acerto

0.7066666666666667

In [28]:
svm_complete_02 = SVC()
svm_complete_02.fit(X_treinamento_02, y_treinamento_02)
previsoes_02 = svm_complete_02.predict(X_teste_02)
taxa_acerto = accuracy_score(y_teste_02, previsoes_02)
taxa_acerto

0.7066666666666667

In [29]:
svm_complete_03 = SVC()
svm_complete_03.fit(X_treinamento_03, y_treinamento_03)
previsoes_03 = svm_complete_03.predict(X_teste_03)
taxa_acerto = accuracy_score(y_teste_03, previsoes_03)
taxa_acerto

0.7066666666666667

### Second algorithm Naive Bayes

In [37]:
naive_bayes_selected_01 = GaussianNB()
naive_bayes_selected_01.fit(X_treinamento_01[:,indices_01[0]], y_treinamento_01)
previsoes = naive_bayes_selected_01.predict(X_teste_01[:,indices_01[0]])
taxa_acerto = accuracy_score(y_teste_01, previsoes)
taxa_acerto

0.7166666666666667

In [38]:
naive_bayes_selected_02 = GaussianNB()
naive_bayes_selected_02.fit(X_treinamento_02[:,indices_02[0]], y_treinamento_02)
previsoes = naive_bayes_selected_02.predict(X_teste_02[:,indices_02[0]])
taxa_acerto = accuracy_score(y_teste_02, previsoes)
taxa_acerto

0.7166666666666667

In [41]:
naive_bayes_selected_03 = GaussianNB()
naive_bayes_selected_03.fit(X_treinamento_03[:,indices_03[0]], y_treinamento_03)
previsoes = naive_bayes_selected_03.predict(X_teste_03[:,indices_03[0]])
taxa_acerto = accuracy_score(y_teste_03, previsoes)
taxa_acerto

0.71

In [39]:
naive_bayes_complete_01 = GaussianNB()
naive_bayes_complete_01.fit(X_treinamento_01, y_treinamento_01)
previsoes = naive_bayes_complete_01.predict(X_teste_01)
taxa_acerto = accuracy_score(y_teste_01, previsoes)
taxa_acerto

0.6933333333333334

In [40]:
naive_bayes_complete_02 = GaussianNB()
naive_bayes_complete_02.fit(X_treinamento_02, y_treinamento_02)
previsoes = naive_bayes_complete_02.predict(X_teste_02)
taxa_acerto = accuracy_score(y_teste_02, previsoes)
taxa_acerto

0.69

In [42]:
naive_bayes_complete_03 = GaussianNB()
naive_bayes_complete_03.fit(X_treinamento_03, y_treinamento_03)
previsoes = naive_bayes_complete_03.predict(X_teste_03)
taxa_acerto = accuracy_score(y_teste_03, previsoes)
taxa_acerto

0.6566666666666666

### The third algorithm using Ranom Forest 

In [44]:
floresta_selected_01 = RandomForestClassifier(n_estimators = 100)
floresta_selected_01.fit(X_treinamento_01[:,indices_01[0]], y_treinamento_01)
previsoes = floresta_selected_01.predict(X_teste_01[:,indices_01[0]])
taxa_acerto = accuracy_score(y_teste_01, previsoes)
taxa_acerto

0.7533333333333333

In [45]:
floresta_selected_02 = RandomForestClassifier(n_estimators = 100)
floresta_selected_02.fit(X_treinamento_02[:,indices_02[0]], y_treinamento_02)
previsoes = floresta_selected_02.predict(X_teste_02[:,indices_02[0]])
taxa_acerto = accuracy_score(y_teste_02, previsoes)
taxa_acerto

0.7433333333333333

In [46]:
floresta_selected_03 = RandomForestClassifier(n_estimators = 100)
floresta_selected_03.fit(X_treinamento_03[:,indices_03[0]], y_treinamento_03)
previsoes = floresta_selected_03.predict(X_teste_03[:,indices_03[0]])
taxa_acerto = accuracy_score(y_teste_03, previsoes)
taxa_acerto

0.7366666666666667

In [47]:
floresta_complete_01 = RandomForestClassifier(n_estimators = 100)
floresta_complete_01.fit(X_treinamento_01, y_treinamento_01)
previsoes = floresta_complete_01.predict(X_teste_01)
taxa_acerto = accuracy_score(y_teste_01, previsoes)
taxa_acerto

0.76

In [48]:
floresta_complete_02 = RandomForestClassifier(n_estimators = 100)
floresta_complete_02.fit(X_treinamento_02, y_treinamento_02)
previsoes = floresta_complete_02.predict(X_teste_02)
taxa_acerto = accuracy_score(y_teste_02, previsoes)
taxa_acerto

0.7433333333333333

In [49]:
floresta_complete_03 = RandomForestClassifier(n_estimators = 100)
floresta_complete_03.fit(X_treinamento_03, y_treinamento_03)
previsoes = floresta_complete_03.predict(X_teste_03)
taxa_acerto = accuracy_score(y_teste_03, previsoes)
taxa_acerto

0.72

### Now I'll try to normalize the 'valor' to see how the algorithm works!

In [36]:
from sklearn import preprocessing

In [38]:
normalized_credito = preprocessing.normalize(credito)

In [60]:
credito['valor'] = normalized_credito[:,3]

In [63]:
'''
Separating predictors and the target value for classification
'''
previsores = credito.iloc[:,0:18].values
classe = credito.iloc[:,18].values

In [67]:
# Spliting data
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(previsores,
                                                                  classe,
                                                                  test_size = 0.3,
                                                                  random_state = 1)

In [68]:
'''
Using a ExtraTreesClassifier to help evaluating 'credito' attributes.
'''
forest = ExtraTreesClassifier()
forest.fit(X_treinamento, y_treinamento)
importancias = forest.feature_importances_
importancias

array([0.06450638, 0.0939424 , 0.0842546 , 0.10090709, 0.06289549,
       0.07678382, 0.06681071, 0.05438602, 0.03379088, 0.06787551,
       0.05293774, 0.03681189, 0.04091602, 0.0380231 , 0.05413231,
       0.02888959, 0.03367193, 0.00846453])

In [69]:
importancias[importancias > 0.05]

array([0.06450638, 0.0939424 , 0.0842546 , 0.10090709, 0.06289549,
       0.07678382, 0.06681071, 0.05438602, 0.06787551, 0.05293774,
       0.05413231])

In [70]:
indices = np.where(importancias > 0.05)

### First algorithm Support Vector Machine

In [71]:
svm_selected = SVC()
svm_selected.fit(X_treinamento[:,indices[0]], y_treinamento)
previsoes = svm_selected.predict(X_teste[:,indices[0]])
taxa_acerto = accuracy_score(y_teste, previsoes)
taxa_acerto

0.7133333333333334

In [72]:
svm_complete = SVC()
svm_complete.fit(X_treinamento, y_treinamento)
previsoes = svm_complete.predict(X_teste)
taxa_acerto = accuracy_score(y_teste, previsoes)
taxa_acerto

0.7066666666666667

#### Creating the second model using Naive Bayes

In [73]:
naive_bayes_selected = GaussianNB()
naive_bayes_selected.fit(X_treinamento[:,indices[0]], y_treinamento)
previsoes = naive_bayes_selected.predict(X_teste[:,indices[0]])
taxa_acerto = accuracy_score(y_teste, previsoes)
taxa_acerto

0.71

In [74]:
naive_bayes_complete = GaussianNB()
naive_bayes_complete.fit(X_treinamento, y_treinamento)
previsoes = naive_bayes_complete.predict(X_teste)
taxa_acerto = accuracy_score(y_teste, previsoes)
taxa_acerto

0.6566666666666666

#### Creating the third model using Ranom Forest 

In [75]:
floresta_selected = RandomForestClassifier(n_estimators = 100)
floresta_selected.fit(X_treinamento[:,indices[0]], y_treinamento)
previsoes = floresta_selected.predict(X_teste[:,indices[0]])
taxa_acerto = accuracy_score(y_teste, previsoes)
taxa_acerto

0.73

In [76]:
floresta_complete = RandomForestClassifier(n_estimators = 100)
floresta_complete.fit(X_treinamento, y_treinamento)
previsoes = floresta_complete.predict(X_teste)
taxa_acerto = accuracy_score(y_teste, previsoes)
taxa_acerto

0.73

In [77]:
credito.head()

,duracao,historicocredito,proposito,valor,investimentos,emprego,tempoparcelamento,estadocivil,fiadorterceiros,residenciadesde,idade,outrosfinanciamentos,habitacao,emprestimoexistente,profissao,dependentes,socioempresa,estrangeiro,status
0,3,2,1,0.999999,2,3.0,2,1,3,2.0,1,3,1.0,1,4,1,0,0,0
1,1,1,2,0.999989,2,4.0,2,4,3,3.0,3,3,1.0,1,3,2,0,0,1
2,3,2,3,0.999999,2,4.0,2,4,4,4.0,2,3,2.0,1,4,2,0,0,1
3,2,3,4,0.999997,2,3.0,3,4,3,4.0,3,3,2.0,2,4,2,0,0,0
4,2,2,2,0.999999,1,3.0,2,4,3,4.0,2,3,2.0,1,3,2,1,0,1


In [78]:
'''
Percist a normalized version of dataset
'''
credito.to_csv('credito_03.csv')

## 3.1 - Creating models using Neural Networking

In [2]:
# Importação das bibliotecas
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier

In [3]:
credito_cleaned = pd.read_csv('credito_01.csv',index_col=[0])
credito_categorical = pd.read_csv('credito_02.csv',index_col=[0])
credito_normalized = pd.read_csv('credito_03.csv',index_col=[0])

In [4]:
credito_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 2 to 565
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   duracao               1000 non-null   int64  
 1   historicocredito      1000 non-null   int64  
 2   proposito             1000 non-null   int64  
 3   valor                 1000 non-null   int64  
 4   investimentos         1000 non-null   int64  
 5   emprego               1000 non-null   float64
 6   tempoparcelamento     1000 non-null   int64  
 7   estadocivil           1000 non-null   int64  
 8   fiadorterceiros       1000 non-null   int64  
 9   residenciadesde       1000 non-null   float64
 10  idade                 1000 non-null   int64  
 11  outrosfinanciamentos  1000 non-null   int64  
 12  habitacao             1000 non-null   float64
 13  emprestimoexistente   1000 non-null   int64  
 14  profissao             1000 non-null   int64  
 15  dependentes           

In [5]:
credito_categorical.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   idcredito             1000 non-null   int64  
 1   duracao               1000 non-null   int64  
 2   historicocredito      1000 non-null   int64  
 3   proposito             1000 non-null   int64  
 4   valor                 1000 non-null   int64  
 5   investimentos         1000 non-null   int64  
 6   emprego               1000 non-null   float64
 7   tempoparcelamento     1000 non-null   int64  
 8   estadocivil           1000 non-null   int64  
 9   fiadorterceiros       1000 non-null   int64  
 10  residenciadesde       1000 non-null   float64
 11  idade                 1000 non-null   int64  
 12  outrosfinanciamentos  1000 non-null   int64  
 13  habitacao             1000 non-null   float64
 14  emprestimoexistente   1000 non-null   int64  
 15  profissao             

In [6]:
'''
Removing idcredito because is not necessary for modeling purposes
'''
credito_categorical.drop(axis=1,columns=['idcredito'],inplace=True)

In [7]:
credito_categorical.head()

,duracao,historicocredito,proposito,valor,investimentos,emprego,tempoparcelamento,estadocivil,fiadorterceiros,residenciadesde,idade,outrosfinanciamentos,habitacao,emprestimoexistente,profissao,dependentes,socioempresa,estrangeiro,status
0,3,2,1,5951,2,3.0,2,1,3,2.0,1,3,1.0,1,4,1,0,0,0
1,1,1,2,2096,2,4.0,2,4,3,3.0,3,3,1.0,1,3,2,0,0,1
2,3,2,3,7882,2,4.0,2,4,4,4.0,2,3,2.0,1,4,2,0,0,1
3,2,3,4,4870,2,3.0,3,4,3,4.0,3,3,2.0,2,4,2,0,0,0
4,2,2,2,9055,1,3.0,2,4,3,4.0,2,3,2.0,1,3,2,1,0,1


In [8]:
credito_normalized.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   duracao               1000 non-null   int64  
 1   historicocredito      1000 non-null   int64  
 2   proposito             1000 non-null   int64  
 3   valor                 1000 non-null   float64
 4   investimentos         1000 non-null   int64  
 5   emprego               1000 non-null   float64
 6   tempoparcelamento     1000 non-null   int64  
 7   estadocivil           1000 non-null   int64  
 8   fiadorterceiros       1000 non-null   int64  
 9   residenciadesde       1000 non-null   float64
 10  idade                 1000 non-null   int64  
 11  outrosfinanciamentos  1000 non-null   int64  
 12  habitacao             1000 non-null   float64
 13  emprestimoexistente   1000 non-null   int64  
 14  profissao             1000 non-null   int64  
 15  dependentes           

#### Training models for cleaned dataset

In [10]:
previsores_cleaned = credito_cleaned.iloc[:,0:18].values
classe_dummy_cleaned = np_utils.to_categorical(credito_cleaned['status'])

In [11]:
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(previsores_cleaned,
                                                                  classe_dummy_cleaned,
                                                                  test_size = 0.3,
                                                                  random_state = 0)

In [12]:
forest = ExtraTreesClassifier()
forest.fit(X_treinamento, y_treinamento)
importancias = forest.feature_importances_
importancias

array([0.10588097, 0.07557865, 0.06998148, 0.09834164, 0.05776722,
       0.06410472, 0.05886869, 0.04767947, 0.03086775, 0.06058183,
       0.09781196, 0.03574431, 0.03845833, 0.03782601, 0.04763787,
       0.02843694, 0.03585531, 0.00857684])

In [13]:
indices = np.where(importancias > 0.05)

In [14]:
modelo_completo = Sequential()
modelo_completo.add(Dense(units = 20, input_dim = 18))
modelo_completo.add(Dense(units = 20))
modelo_completo.add(Dense(units = 10))
modelo_completo.add(Dense(units = 2, activation = 'softmax'))

In [15]:
modelo_completo.compile(optimizer = 'adam', 
               loss = 'categorical_crossentropy',
               metrics = ['accuracy'])
modelo_completo.fit(X_treinamento, y_treinamento, epochs = 1000, verbose=False,
           validation_data = (X_teste, y_teste))

In [16]:
# evaluate the model
_, train_acc = modelo_completo.evaluate(X_treinamento, y_treinamento, verbose=0)
_, test_acc = modelo_completo.evaluate(X_teste, y_teste, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

Train: 0.693, Test: 0.770


In [17]:
indices[0]

array([ 0,  1,  2,  3,  4,  5,  6,  9, 10])

In [19]:
modelo_selected = Sequential()
modelo_selected.add(Dense(units = 20, input_dim = len(indices[0])))
modelo_selected.add(Dense(units = 20))
modelo_selected.add(Dense(units = 10))
modelo_selected.add(Dense(units = 2, activation = 'softmax'))

In [20]:
modelo_selected.compile(optimizer = 'adam', 
               loss = 'categorical_crossentropy',
               metrics = ['accuracy'])
modelo_selected.fit(X_treinamento[:,indices[0]], y_treinamento, epochs = 1000, verbose=False,
           validation_data = (X_teste[:,indices[0]], y_teste))

In [21]:
# evaluate the model
_, train_acc_2 = modelo_selected.evaluate(X_treinamento[:,indices[0]], y_treinamento, verbose=0)
_, test_acc_2 = modelo_selected.evaluate(X_teste[:,indices[0]], y_teste, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc_2, test_acc_2))

Train: 0.671, Test: 0.713


#### Training models for categorical dataset

In [15]:
previsores_categorical = credito_categorical.iloc[:,0:18].values
classe_dummy_categorical = np_utils.to_categorical(credito_categorical['status'])

In [16]:
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(previsores_categorical,
                                                                  previsores_categorical,
                                                                  test_size = 0.3,
                                                                  random_state = 0)

In [22]:
modelo_completo.compile(optimizer = 'adam', 
               loss = 'categorical_crossentropy',
               metrics = ['accuracy'])
modelo_completo.fit(X_treinamento, y_treinamento, epochs = 1000, verbose=False,
           validation_data = (X_teste, y_teste))

In [24]:
# evaluate the model
_, train_acc = modelo_completo.evaluate(X_treinamento, y_treinamento, verbose=0)
_, test_acc = modelo_completo.evaluate(X_teste, y_teste, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

Train: 0.690, Test: 0.760


In [25]:
forest = ExtraTreesClassifier()
forest.fit(X_treinamento, y_treinamento)
importancias = forest.feature_importances_
importancias

array([0.10402433, 0.07272384, 0.07207744, 0.10326271, 0.05942286,
       0.06426545, 0.06071391, 0.04783915, 0.02934317, 0.06071454,
       0.09460328, 0.0371871 , 0.03766895, 0.03550478, 0.04858451,
       0.02854074, 0.03494028, 0.00858298])

In [26]:
indices = np.where(importancias > 0.05)

In [27]:
indices[0]

array([ 0,  1,  2,  3,  4,  5,  6,  9, 10])

In [28]:
modelo_selected = Sequential()
modelo_selected.add(Dense(units = 20, input_dim = len(indices[0])))
modelo_selected.add(Dense(units = 20))
modelo_selected.add(Dense(units = 10))
modelo_selected.add(Dense(units = 2, activation = 'softmax'))

In [29]:
modelo_selected.compile(optimizer = 'adam', 
               loss = 'categorical_crossentropy',
               metrics = ['accuracy'])
modelo_selected.fit(X_treinamento[:,indices[0]], y_treinamento, epochs = 1000, verbose=False,
           validation_data = (X_teste[:,indices[0]], y_teste))

In [30]:
# evaluate the model
_, train_acc_2 = modelo_selected.evaluate(X_treinamento[:,indices[0]], y_treinamento, verbose=0)
_, test_acc_2 = modelo_selected.evaluate(X_teste[:,indices[0]], y_teste, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc_2, test_acc_2))

Train: 0.683, Test: 0.750


In [11]:
# threshold for predictions
#previsoes = modelo.predict(X_teste)
#previsoes = (previsoes > 0.5)
#previsoes

In [31]:
previsores_normalized = credito_normalized.iloc[:,0:18].values
classe_dummy_normalized = np_utils.to_categorical(credito_normalized['status'])

In [32]:
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(previsores_normalized,
                                                                  classe_dummy_normalized,
                                                                  test_size = 0.3,
                                                                  random_state = 0)

In [33]:
modelo_completo.compile(optimizer = 'adam', 
               loss = 'categorical_crossentropy',
               metrics = ['accuracy'])
modelo_completo.fit(X_treinamento, y_treinamento, epochs = 1000, verbose=False,
           validation_data = (X_teste, y_teste))

In [34]:
# evaluate the model
_, train_acc = modelo_completo.evaluate(X_treinamento, y_treinamento, verbose=0)
_, test_acc = modelo_completo.evaluate(X_teste, y_teste, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

Train: 0.701, Test: 0.770


In [35]:
forest = ExtraTreesClassifier()
forest.fit(X_treinamento, y_treinamento)
importancias = forest.feature_importances_
importancias

array([0.06230774, 0.0761877 , 0.08374255, 0.10229968, 0.06838735,
       0.07350471, 0.06595665, 0.04850275, 0.03382278, 0.07393018,
       0.05649689, 0.03845689, 0.04230674, 0.03947911, 0.05502729,
       0.02786512, 0.04233639, 0.00938948])

In [36]:
indices = np.where(importancias > 0.05)

In [37]:
indices[0]

array([ 0,  1,  2,  3,  4,  5,  6,  9, 10, 14])

In [38]:
modelo_selected = Sequential()
modelo_selected.add(Dense(units = 20, input_dim = len(indices[0])))
modelo_selected.add(Dense(units = 20))
modelo_selected.add(Dense(units = 10))
modelo_selected.add(Dense(units = 2, activation = 'softmax'))

In [39]:
modelo_selected.compile(optimizer = 'adam', 
               loss = 'categorical_crossentropy',
               metrics = ['accuracy'])
modelo_selected.fit(X_treinamento[:,indices[0]], y_treinamento, epochs = 1000, verbose=False,
           validation_data = (X_teste[:,indices[0]], y_teste))

In [40]:
# evaluate the model
_, train_acc_2 = modelo_selected.evaluate(X_treinamento[:,indices[0]], y_treinamento, verbose=0)
_, test_acc_2 = modelo_selected.evaluate(X_teste[:,indices[0]], y_teste, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc_2, test_acc_2))

Train: 0.696, Test: 0.757


# Conclusion

Based in previews results I'll publish in production the model was created using Neural Network with all attributes from a categorical model that acheive 77% of accuracy in test data.